In [1]:
import numpy as np 
import pandas as pd 


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
!pip install datasets
!pip install transformers
!pip install peft
!pip install evaluate
!pip install -U datasets==2.20.0 pyarrow==15.0.2 transformers==4.44.2 evaluate==0.4.2 --no-cache-dir
!pip install -q datasets evaluate accelerate scikit-learn pandas matplotlib
!pip install -U "transformers>=4.41" accelerate safetensors
!pip install -U bitsandbytes


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 149.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 104.5 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 321.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.3
    Uninstalling transformers-4.57.3:
      Successfully uninstalled transformers-4.57.3
  Using cached transformers-4.57.3-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
Using cached transformers-4.57.3-py3-none-any.whl (12.0 MB)
Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tok

In [21]:
!pip install --upgrade --no-cache-dir git+https://github.com/Shannu3766/Cosine-similarity.git
# !pip uninstall adalora_bi_eq13 -y

  Cloning https://github.com/Shannu3766/Cosine-similarity.git to /tmp/pip-req-build-pife8ffj
  Running command git clone --filter=blob:none --quiet https://github.com/Shannu3766/Cosine-similarity.git /tmp/pip-req-build-pife8ffj
  Resolved https://github.com/Shannu3766/Cosine-similarity.git to commit 1cb3f52a6fdbe414d56e9bc842d579c329e07203
  Preparing metadata (setup.py) ... done
  Created wheel for adalora_bi_eq13: filename=adalora_bi_eq13-0.1.0-py3-none-any.whl size=7116 sha256=f17196452cf69a7f9a81d613cafe4bb25ed3c46fd5b10992cc4eaf31ae6f2000
  Stored in directory: /tmp/pip-ephem-wheel-cache-q_uahhsc/wheels/7e/9d/f3/13898734a779d7b427419fa0ff63bf393a2a18fb8f70a46662
Successfully built adalora_bi_eq13


In [1]:
import warnings
import os
import logging
os.environ["WANDB_DISABLED"] = "true"
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "true"
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings("ignore")

In [2]:
import os
import torch
import numpy as np
import evaluate
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)
from peft import get_peft_model, LoraConfig, TaskType
os.environ["WANDB_DISABLED"] = "true"


In [3]:
MODEL_ID = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
TASK = "rte"
LR = 2e-4
NUM_EPOCHS = 3
BATCH_SIZE = 8 
MAX_LEN = 256  
rank= 8

In [4]:
import time
def load_dataset_with_retry(path, name=None, max_retries=None, wait=2):

    attempt = 0

    while True:
        try:
            if name is not None:
                ds = load_dataset(path, name)
            else:
                ds = load_dataset(path)

            print(f"Dataset loaded successfully after {attempt} attempts.")
            return ds

        except Exception as e:
            attempt += 1
            wait_time = wait * min(5, attempt)  

            print(f"[Attempt {attempt}] Failed to load dataset: {e}")
            print(f"Retrying in {wait_time} seconds...\n")

            if max_retries is not None and attempt >= max_retries:
                print("Max retries reached. Raising error.")
                raise e

            time.sleep(wait_time)

dataset = load_dataset_with_retry("glue", TASK)


Dataset loaded successfully after 0 attempts.


In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 2490
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 277
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3000
    })
})

In [6]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)

In [7]:
from transformers import DataCollatorWithPadding

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

MAX_LEN = 256 


def clean_text(s):
    if s is None:
        return ""

    return " ".join(str(s).strip().split())


def preprocess_function(examples):
    premises = [clean_text(p) for p in examples["sentence1"]]
    hypotheses = [clean_text(h) for h in examples["sentence2"]]

    inputs = [
        (
            f"Premise: {p}\n"
            f"Hypothesis: {h}\n"
            f"Question: Does the premise entail the hypothesis?\n"
            f"Answer:"
        )
        for p, h in zip(premises, hypotheses)
    ]

    tokenized = tokenizer(
        inputs,
        truncation=True,
        max_length=MAX_LEN,
        padding=False,
    )
    return tokenized


encoded_dataset = dataset.map(preprocess_function, batched=True)

encoded_dataset = encoded_dataset.remove_columns(["sentence1", "sentence2", "idx"])

if "label" in encoded_dataset["train"].column_names and "labels" not in encoded_dataset["train"].column_names:
    encoded_dataset = encoded_dataset.rename_column("label", "labels")

encoded_dataset.set_format("torch")

data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    padding="longest",
)


In [8]:
print("Model started Loading")

Model started Loading


In [9]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_ID,
    num_labels=2,
    device_map="auto",
    torch_dtype=torch.bfloat16, 
    trust_remote_code=True
)

print("Model Loaded")
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False 

`torch_dtype` is deprecated! Use `dtype` instead!
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at TinyLlama/TinyLlama-1.1B-Chat-v1.0 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model Loaded


In [10]:
print("Model Loaded")
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False 

Model Loaded


In [11]:
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=rank,                    
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=[   
        "q_proj", "k_proj", "v_proj", "o_proj", 
        "gate_proj", "up_proj", "down_proj"
    ]
)

In [12]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainable params: 6,311,936 || all params: 1,040,828,416 || trainable%: 0.6064


In [13]:
from peft.tuners.lora import LoraLayer

def get_lora_module_names(peft_model):
    names = []
    for name, module in peft_model.named_modules():
        if isinstance(module, LoraLayer):
            names.append(name)
    return names

lora_names = get_lora_module_names(model)   
print(f"🔢 Number of LoRA modules: {len(lora_names)}\n")

num_lora_modules = len(lora_names)
TOTAL_RANK_BUDGET = rank*num_lora_modules  

avg_rank_per_module = TOTAL_RANK_BUDGET / num_lora_modules
print(
    f"💡 With TOTAL_RANK_BUDGET={TOTAL_RANK_BUDGET} over "
    f"{num_lora_modules} modules, avg rank ≈ {avg_rank_per_module:.2f}"
)


🔢 Number of LoRA modules: 154

💡 With TOTAL_RANK_BUDGET=1232 over 154 modules, avg rank ≈ 8.00


In [14]:
training_args = TrainingArguments(
    output_dir=f"{MODEL_ID}-RTE-v2",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    weight_decay=0.01,
    logging_steps=20,
    metric_for_best_model="accuracy",
    fp16=False,             
    bf16=True,      
    report_to="none",
)

In [15]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
from adaptive_lora_gradient.callbacks import AdaptiveLoRACallback
import torch



if "label" in encoded_dataset["train"].column_names and "labels" not in encoded_dataset["train"].column_names:
    encoded_dataset = encoded_dataset.rename_column("label", "labels")

keep_cols = {"input_ids", "attention_mask", "labels"}

for split in encoded_dataset.keys():
    drop_cols = [c for c in encoded_dataset[split].column_names if c not in keep_cols]
    if drop_cols:
        encoded_dataset[split] = encoded_dataset[split].remove_columns(drop_cols)

encoded_dataset.set_format(type="torch")

data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    padding="longest",
)

train_dataset = encoded_dataset["train"]
eval_dataset = encoded_dataset["validation"]

EVAL_BATCH_SIZE = getattr(training_args, "per_device_eval_batch_size", 8)

val_dataloader = DataLoader(
    eval_dataset,
    batch_size=EVAL_BATCH_SIZE,
    shuffle=False,
    collate_fn=data_collator,
    pin_memory=torch.cuda.is_available(),
    num_workers=2,
)


adaptive_callback = AdaptiveLoRACallback(
    total_rank=TOTAL_RANK_BUDGET,
    val_dataloader=val_dataloader,
    score_smoothing_beta=0.8,  
    update_interval=2,         
    warmup_epochs=0,           
    cooldown_epochs=1,         
    min_rank=4,
    tau=0.9
)
print("✅ Adaptive LoRA callback ready.")

✅ Adaptive LoRA callback ready.


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[adaptive_callback],
)

print("🚀 Starting Training...")
trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.


🚀 Starting Training...

--- AdaptiveLoRA: Preparing ranks for Epoch 1 ---
Computing BI importance scores (pre-training)...


Computing Importance:   0%|          | 0/35 [00:00<?, ?it/s]

In [ ]:
print("📊 Final Evaluation:")
results = trainer.evaluate()
print(results)

📊 Final Evaluation:


{'eval_loss': 1.3090718984603882, 'eval_accuracy': 0.8014440433212996, 'eval_runtime': 52.2391, 'eval_samples_per_second': 5.303, 'eval_steps_per_second': 0.67, 'epoch': 5.0}
